In [15]:
import pandas as pd

def format_excel_sheets(file_path):
    # Read the Excel file
    excel_file = pd.ExcelFile(file_path)
    
    # Dictionary to store formatted output for each sheet
    output = {}
    
    # Loop through each sheet
    for sheet_name in excel_file.sheet_names:
        # Read the sheet into a DataFrame
        df = pd.read_excel(file_path, sheet_name=sheet_name)
        # List to store formatted strings for this sheet
        sheet_output = []
        
        # Get column names
        col_names = df.columns
        
        # Loop through each row
        for index, row in df.iterrows():
            # Create a formatted string for the row
            row_strings = []
            for col_name, value in zip(col_names, row):
                # Handle NaN values
                if pd.notna(value):
                    value_str = str(value)
                    row_strings.append(f"{col_name}: {value_str}")
            # Join all column entries for this row
            sheet_output.append(", ".join(row_strings))
        # Store the formatted strings for this sheet
        output[sheet_name] = sheet_output
    
    return output

# Example usage
def get_alias_dictionary(file_path="alias.xlsx"):
    try:
        formatted_sheets = format_excel_sheets(file_path)
        
        return_string = ""
        # Print formatted output for each sheet
        for sheet_name, rows in formatted_sheets.items():
            return_string += f"Sheet: {sheet_name}\n"
            for row in rows:
                return_string += f"{row}\n"
            return_string += "\n"  # Empty line between sheets
        return return_string    
    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found.")
    except Exception as e:
        print(f"Error: {str(e)}")
        

In [25]:
ALIAS_HANDLE_PROMPT = """You are a specialized AI module within a financial Excel chatbot. Your primary function is to act as a **Query Normalizer**. Your task is to make user queries more meaningful by enriching them with information from an alias dictionary, without altering the original terms needed for data retrieval.

Your goal is to identify all known aliases within an `Initial Query` and append their corresponding full names or alternative identifiers in parentheses. **You must not replace the original alias.** This is a critical rule because the underlying Excel data often contains the alias itself (e.g., the column is named "CP", not "Chi phí").

You will be provided with two inputs:
1.  `Alias Dictionary`: A structured list of known aliases and their corresponding full or official terms.
2.  `Initial Query`: The raw, unprocessed question from the end-user.

Your sole output should be the `Enriched Query`.

**Rules for Enrichment:**

1.  **Augment, Don't Replace:** For every term in the `Initial Query` that is found as an alias in the `Alias Dictionary`, you must keep the original term and append its context in parentheses right after it.
2.  **Annotation Format:**
    *   For simple abbreviations, use the format: `Alias(Full Term)`. Example: `Chi phí(CP)`.
    *   For entities with multiple identifiers (like a nickname and a full name), use the format: `Alias(Identifier 1 - Identifier 2)`. Example: `Telemor(VTL - Viettel Timor Leste) or Viettel Timor Leste(VTL - Telemor)`.
3.  **No Match:** If a word in the query is not found in the Alias Dictionary, leave it completely unchanged.

**Example**
### Alias Dictionary:
Sheet: Từ viết tắt
TT: 1, Viết tắt: CP, Từ đầy đủ: Chi phí, Từ viết tắt khác: C.phí
TT: 2, Viết tắt: DA, Từ đầy đủ: Dự án, Từ viết tắt khác: D.án
TT: 3, Viết tắt: DT, Từ đầy đủ: Doanh thu

Sheet: Khách hàng
TT: 1, Viết tắt: Deeeplabs, Tên đầy đủ: Deeeplabs Pte Ltd
TT: 2, Viết tắt: SkyIQ, Tên đầy đủ: SKYIQ PTE.LTD
TT: 3, Viết tắt: VTL, Tên thường gọi: Telemor, Tên đầy đủ: Viettel Timor Leste

### Initial Query
CP cho sản xuất là bao nhiêu năm 2022?
### Enriched Query
CP(Chi phí) cho sản xuất là bao nhiêu năm 2022?

### Initial Query
CP cho DA tháng 1 và tháng 4 của Telemor là bao nhiêu?
### Enriched Query
CP(Chi phí) cho DA(Dự án) tháng 1 và tháng 4 của Telemor(VTL - Viettel Timor Leste) là bao nhiêu?

### Initial Query
tổng doanh thu của deeeplabs trong quý 1?
### Enriched Query
tổng doanh thu(DT) của deeeplabs(Deeeplabs Pte Ltd) trong quý 1?

### Initial Query
So sánh Doanh Thu của SkyIQ và Viettel Timor Leste.
### Enriched Query
So sánh Doanh Thu(DT) của SkyIQ(SKYIQ PTE.LTD) và Viettel Timor Leste(Telemor-VTL).

Now deal with the following case:
### Alias Dictionary:
{alias_dictionary}

### Initial Query
{user_query}
### Enriched Query
"""

In [26]:
import os
import getpass
import pandas as pd
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage
import openpyxl
import dotenv
dotenv.load_dotenv()

# Initialize the Gemini 2.5 Flash model
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-preview-04-17",
    google_api_key=os.getenv("GOOGLE_API_KEY_1"),
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [27]:
alias_dictionary = get_alias_dictionary()
user_query = "Doanh thu của Trung tâm Trí tuệ nhân tạo Viettel vào năm 2023"

In [28]:
prompt = ALIAS_HANDLE_PROMPT.format(alias_dictionary=alias_dictionary, user_query=user_query)
message = HumanMessage(content=prompt)

response = llm.invoke([message])
print(response.content)

Doanh thu(DT) của Trung tâm Trí tuệ nhân tạo Viettel(VAI - Viettel AI) vào năm 2023
